In [3]:
import pandas as pd

In [5]:
# Load the CSV file into a DataFrame bolig_laeger_st_DST
csv_file_path = 'outputdata/NW_Merged.csv'   # Ret sti
data = pd.read_csv(csv_file_path , index_col=1)

data_imputed = data[[
    'Rooms_imputed',
    'Toilets_imputed',
    'Karakter_imputed',
    'indikator_imputed',
    'Socioeconomic_Ref_Imputed',
    'sum_imputed'
]]

data_for_model=data.drop(['Schoolname', 
                        'Average_Grade',
                        'Indikatorsvar',
                        'Matched_Kommune',
                        'link', 
                        'Unnamed: 0.1',
                        'Rooms_imputed',
                        'Toilets_imputed',
                        'Karakter_imputed',
                        'indikator_imputed',
                        'Socioeconomic_Ref_Imputed',
                        'sum_imputed'
                                        
                                          ], axis=1)


In [7]:
#Data for landkort

data_nodup = data[~data.index.duplicated(keep='first')]
house_count_df = data_nodup.groupby('Postal code').size().reset_index(name='NumberOfHouses')
house_count_df.to_csv('outputdata/Post_count.csv', index=False)

In [9]:
pd.set_option('display.max_column', None)
data_uden_missing = data_for_model.dropna()
data_uden_missing = data_uden_missing[data_uden_missing['Boligtype'].str.strip() != 'Fritidsbolig']
data_uden_missing.to_csv('outputdata/data_uden_missing.csv', index=False)

In [11]:
#Postnummer dummies
df=data_uden_missing

# Let's correct the approach by directly assigning the location categories without using pd.cut, 
# and then create the dummy variables

# Initialize the Location_Category column with None values
df['Location_Category'] = None

# Assign values based on the postal code ranges

#KØBENHAVN
df.loc[(df['Postal code'] >= 1000) & (df['Postal code'] <= 1499), 'Location_Category'] = 'KBH_K'
df.loc[(df['Postal code'] >= 1500) & (df['Postal code'] <= 1799), 'Location_Category'] = 'KBH_V'
df.loc[(df['Postal code'] == 2100) | (df['Postal code'] == 2150), 'Location_Category'] = 'KBH_Ø'
df.loc[(df['Postal code'] == 2000), 'Location_Category'] = 'FRED'
df.loc[(df['Postal code'] >= 1800) & (df['Postal code'] <= 1974), 'Location_Category'] = 'FRED_C'
df.loc[(df['Postal code'] == 2200) | (df['Postal code'] == 2400), 'Location_Category'] = 'KBH_N'
df.loc[(df['Postal code'] == 2300) | (df['Postal code'] == 2450), 'Location_Category'] = 'KBH_S_SV'
df.loc[(df['Postal code'] == 2500), 'Location_Category'] = 'Valby'
df.loc[(df['Postal code'] == 2900), 'Location_Category'] = 'HEL'

#ÅRHUS, AALBORG, ODENSE
df.loc[(df['Postal code'] == 8000), 'Location_Category'] = 'AAR_C'
df.loc[(df['Postal code'] == 8200), 'Location_Category'] = 'AAR_N'
df.loc[(df['Postal code'] == 8210), 'Location_Category'] = 'AAR_V'
df.loc[(df['Postal code'] == 9000), 'Location_Category'] = 'AAL_C'
df.loc[(df['Postal code'] == 9200) | (df['Postal code'] == 9210) | (df['Postal code'] == 9220), 'Location_Category'] = 'AAL'
df.loc[(df['Postal code'] == 5000), 'Location_Category'] = 'ODE_C'
df.loc[(df['Postal code'] >= 5200) & (df['Postal code'] <= 5270), 'Location_Category'] = 'ODE'


df.loc[(df['Postal code'] >= 2800) & (df['Postal code'] <= 2850) | (df['Postal code'] >= 2920) & (df['Postal code'] <= 2990), 'Location_Category'] = 'Whisky'
# Create dummy variables for these categories
df_dummies = pd.get_dummies(df['Location_Category'])

# Add the dummy variables to the original dataframe
df = pd.concat([df, df_dummies], axis=1)

data_uden_missing=df.drop(['Location_Category', 'Postal code'], axis=1)

In [13]:
import pandas as pd

def create_dummies_with_reference(df, columns_with_references):
    """
    Creates dummy variables with a specified reference category.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    columns_with_references (dict): A dictionary where keys are column names and values are the reference categories.

    Returns:
    pd.DataFrame: DataFrame with dummy variables created, with the specified reference categories dropped.
    """
    df = df.copy()  # Make a copy of the DataFrame to avoid modifying the original

    for column, reference in columns_with_references.items():
        # Ensure the column is categorical
        df[column] = df[column].astype('category')
        
        # Check if the reference exists in the column
        if reference not in df[column].cat.categories:
            raise ValueError(f"The reference category '{reference}' does not exist in the column '{column}'")
        
        # Reorder categories to place the reference first
        df[column] = df[column].cat.reorder_categories(
            [reference] + [cat for cat in df[column].cat.categories if cat != reference], 
            ordered=True
        )
    
    # Generate dummies with the first category (the reference) dropped
    df = pd.get_dummies(df, columns=columns_with_references.keys(), drop_first=True)
    
    return df

columns_with_references = {
    'Boligtype': 'Villa',
    'Energy-classification':' D',
    'Radon': 'Mellem',
    'Kommune': 'herning kommune',
    '1 årig - Socioøkonomisk Reference Forskel': 'På Niveau'
}

# Apply the function
data_uden_missing = create_dummies_with_reference(data_uden_missing, columns_with_references)

In [15]:
data_uden_missing

,Price,Distance to school,Trivsel,Kommuneskat,Grundskyldspromillen,Liggetid,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Elevator,Altan,Terasse,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand,Antal_laeger,Stationer,Karakter,AAL,AAL_C,AAR_C,AAR_N,AAR_V,FRED,FRED_C,HEL,KBH_K,KBH_N,KBH_S_SV,KBH_V,KBH_Ø,ODE,ODE_C,Valby,Whisky,Boligtype_Ejerlejlighed,Boligtype_Landejendom,Boligtype_Rækkehus,Boligtype_Villalejlighed,Energy-classification_ A,Energy-classification_ A2,Energy-classification_ A2010,Energy-classification_ A2015,Energy-classification_ A2020,Energy-classification_ B,Energy-classification_ C,Energy-classification_ E,Energy-classification_ F,Energy-classification_ G,Energy-classification_fejl,Radon_Høj,Radon_Lav,Radon_Meget høj,Radon_Meget lav,Radon_Ukendt,Kommune_aabenraa kommune,Kommune_aalborg kommune,Kommune_aarhus kommune,Kommune_albertslund kommune,Kommune_allerød kommune,Kommune_assens kommune,Kommune_ballerup kommune,Kommune_billund kommune,Kommune_bornholm kommune,Kommune_brøndby kommune,Kommune_brønderslev kommune,Kommune_dragør kommune,Kommune_egedal kommune,Kommune_esbjerg kommune,Kommune_faaborg-midtfyn kommune,Kommune_fanø kommune,Kommune_favrskov kommune,Kommune_faxe kommune,Kommune_fredensborg kommune,Kommune_fredericia kommune,Kommune_frederiksberg kommune,Kommune_frederikshavn kommune,Kommune_frederikssund kommune,Kommune_furesø kommune,Kommune_gentofte kommune,Kommune_gladsaxe kommune,Kommune_glostrup kommune,Kommune_greve kommune,Kommune_gribskov kommune,Kommune_guldborgsund kommune,Kommune_haderslev kommune,Kommune_halsnæs kommune,Kommune_hedensted kommune,Kommune_helsingør kommune,Kommune_herlev kommune,Kommune_hillerød kommune,Kommune_hjørring kommune,Kommune_holbæk kommune,Kommune_holstebro kommune,Kommune_horsens kommune,Kommune_hvidovre kommune,Kommune_høje-taastrup kommune,Kommune_hørsholm kommune,Kommune_ikast-brande kommune,Kommune_ishøj kommune,Kommune_jammerbugt kommune,Kommune_kalundborg kommune,Kommune_kerteminde kommune,Kommune_kolding kommune,Kommune_københavns kommune,Kommune_køge kommune,Kommune_langeland kommune,Kommune_lejre kommune,Kommune_lemvig kommune,Kommune_lolland kommune,Kommune_lyngby-taarbæk kommune,Kommune_læsø kommune,Kommune_mariagerfjord kommune,Kommune_middelfart kommune,Kommune_morsø kommune,Kommune_norddjurs kommune,Kommune_nordfyns kommune,Kommune_nyborg kommune,Kommune_næstved kommune,Kommune_odder kommune,Kommune_odense kommune,Kommune_odsherred kommune,Kommune_randers kommune,Kommune_rebild kommune,Kommune_ringkøbing-skjern kommune,Kommune_ringsted kommune,Kommune_roskilde kommune,Kommune_rudersdal kommune,Kommune_rødovre kommune,Kommune_samsø kommune,Kommune_silkeborg kommune,Kommune_skanderborg kommune,Kommune_skive kommune,Kommune_slagelse kommune,Kommune_solrød kommune,Kommune_sorø kommune,Kommune_stevns kommune,Kommune_struer kommune,Kommune_svendborg kommune,Kommune_syddjurs kommune,Kommune_sønderborg kommune,Kommune_thisted kommune,Kommune_tårnby kommune,Kommune_tønder kommune,Kommune_vallensbæk kommune,Kommune_varde kommune,Kommune_vejen kommune,Kommune_vejle kommune,Kommune_vesthimmerlands kommune,Kommune_viborg kommune,Kommune_vordingborg kommune,Kommune_ærø kommune,1 årig - Socioøkonomisk Reference Forskel_Bedre end forventet,1 årig - Socioøkonomisk Reference Forskel_Dårligere end forventet
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Mellemtoftevej 36,2600",3995000.0,0.391,3.502026,24.60,25.00,1.0,128.0,388.0,2.889,5.0,1947,1.0,0,0,0,2660,1096,407100.0,47.4,23899.0,0.738859,0.055232,0.205908,13.3,6.0,0.0,6.896407,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,F

In [17]:
non_numeric_columns = data_uden_missing.select_dtypes(include=['object']).columns
non_numeric_columns

# Force all columns to numeric, coercing errors to NaN
data_numeric = data_uden_missing.apply(pd.to_numeric, errors='coerce')

# Identify columns that could not be converted to numeric and remain non-numeric
non_numeric_cols = data_numeric.columns[data_numeric.dtypes == 'object']

# Display any non-numeric columns
non_numeric_cols

data_numeric = data_numeric.astype('float32')


In [19]:
data_numeric.to_csv('outputdata/data_ready.csv')
data_numeric

,Price,Distance to school,Trivsel,Kommuneskat,Grundskyldspromillen,Liggetid,m2,Grund,Ejerudgift,Rooms,Build,Toilets,Elevator,Altan,Terasse,antal anmeldelser,antal sigtelser,Familiernes gennemsnitlige indkomst indkomst,"m2 parker, sportsanlæg og rekreative omr pr indbygger",indbyggertal,andel_dansk_opr,andel_vestlig_opr,andel_ikke_vestlig_opr,Gennemsnitlig_pendligafstand,Antal_laeger,Stationer,Karakter,AAL,AAL_C,AAR_C,AAR_N,AAR_V,FRED,FRED_C,HEL,KBH_K,KBH_N,KBH_S_SV,KBH_V,KBH_Ø,ODE,ODE_C,Valby,Whisky,Boligtype_Ejerlejlighed,Boligtype_Landejendom,Boligtype_Rækkehus,Boligtype_Villalejlighed,Energy-classification_ A,Energy-classification_ A2,Energy-classification_ A2010,Energy-classification_ A2015,Energy-classification_ A2020,Energy-classification_ B,Energy-classification_ C,Energy-classification_ E,Energy-classification_ F,Energy-classification_ G,Energy-classification_fejl,Radon_Høj,Radon_Lav,Radon_Meget høj,Radon_Meget lav,Radon_Ukendt,Kommune_aabenraa kommune,Kommune_aalborg kommune,Kommune_aarhus kommune,Kommune_albertslund kommune,Kommune_allerød kommune,Kommune_assens kommune,Kommune_ballerup kommune,Kommune_billund kommune,Kommune_bornholm kommune,Kommune_brøndby kommune,Kommune_brønderslev kommune,Kommune_dragør kommune,Kommune_egedal kommune,Kommune_esbjerg kommune,Kommune_faaborg-midtfyn kommune,Kommune_fanø kommune,Kommune_favrskov kommune,Kommune_faxe kommune,Kommune_fredensborg kommune,Kommune_fredericia kommune,Kommune_frederiksberg kommune,Kommune_frederikshavn kommune,Kommune_frederikssund kommune,Kommune_furesø kommune,Kommune_gentofte kommune,Kommune_gladsaxe kommune,Kommune_glostrup kommune,Kommune_greve kommune,Kommune_gribskov kommune,Kommune_guldborgsund kommune,Kommune_haderslev kommune,Kommune_halsnæs kommune,Kommune_hedensted kommune,Kommune_helsingør kommune,Kommune_herlev kommune,Kommune_hillerød kommune,Kommune_hjørring kommune,Kommune_holbæk kommune,Kommune_holstebro kommune,Kommune_horsens kommune,Kommune_hvidovre kommune,Kommune_høje-taastrup kommune,Kommune_hørsholm kommune,Kommune_ikast-brande kommune,Kommune_ishøj kommune,Kommune_jammerbugt kommune,Kommune_kalundborg kommune,Kommune_kerteminde kommune,Kommune_kolding kommune,Kommune_københavns kommune,Kommune_køge kommune,Kommune_langeland kommune,Kommune_lejre kommune,Kommune_lemvig kommune,Kommune_lolland kommune,Kommune_lyngby-taarbæk kommune,Kommune_læsø kommune,Kommune_mariagerfjord kommune,Kommune_middelfart kommune,Kommune_morsø kommune,Kommune_norddjurs kommune,Kommune_nordfyns kommune,Kommune_nyborg kommune,Kommune_næstved kommune,Kommune_odder kommune,Kommune_odense kommune,Kommune_odsherred kommune,Kommune_randers kommune,Kommune_rebild kommune,Kommune_ringkøbing-skjern kommune,Kommune_ringsted kommune,Kommune_roskilde kommune,Kommune_rudersdal kommune,Kommune_rødovre kommune,Kommune_samsø kommune,Kommune_silkeborg kommune,Kommune_skanderborg kommune,Kommune_skive kommune,Kommune_slagelse kommune,Kommune_solrød kommune,Kommune_sorø kommune,Kommune_stevns kommune,Kommune_struer kommune,Kommune_svendborg kommune,Kommune_syddjurs kommune,Kommune_sønderborg kommune,Kommune_thisted kommune,Kommune_tårnby kommune,Kommune_tønder kommune,Kommune_vallensbæk kommune,Kommune_varde kommune,Kommune_vejen kommune,Kommune_vejle kommune,Kommune_vesthimmerlands kommune,Kommune_viborg kommune,Kommune_vordingborg kommune,Kommune_ærø kommune,1 årig - Socioøkonomisk Reference Forskel_Bedre end forventet,1 årig - Socioøkonomisk Reference Forskel_Dårligere end forventet
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Mellemtoftevej 36,2600",3995000.0,0.391,3.502026,24.600000,25.000000,1.0,128.0,388.0,2.889,5.0,1947.0,1.0,0.0,0.0,0.0,2660.0,1096.0,407100.0,47.400002,23899.0,0.738859,0.055232,0.205908,13.300000,6.0,0.0,6.896407,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [21]:
#OLS
import statsmodels.api as sm

# Redefine the dependent and independent variables
Y = data_numeric['Price']
X = data_numeric.drop(columns=['Price'])

# Add a constant to the independent variables matrix
X = sm.add_constant(X)

# Perform OLS regression with robust standard errors
model = sm.OLS(Y, X)
results = model.fit(cov_type='HC3')  # HC3 provides robust standard errors

# Display the regression summary
ols_summary_clean = results.summary()
ols_summary_clean


/Applications/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 162, but rank is 148
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.642
Method:                 Least Squares   F-statistic:                     83.43
Date:                Thu, 22 Aug 2024   Prob (F-statistic):               0.00
Time:                        16:06:35   Log-Likelihood:            -3.1825e+05
No. Observations:               19848   AIC:                         6.368e+05
Df Residuals:                   19697   BIC:                         6.380e+05
Df Model:                         150                                         
Covariance Type:                  HC3                                         
=====================================================================================================================================
                                                                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------------------
const                                                             -4.941e+05   6.55e+04     -7.541      0.000   -6.23e+05   -3.66e+05
Distance to school                                                -7.531e+04   7126.640    -10.568      0.000   -8.93e+04   -6.13e+04
Trivsel                                                            2.965e+06   2.22e+05     13.360      0.000    2.53e+06     3.4e+06
Kommuneskat                                                       -5.486e+05   4.42e+04    -12.418      0.000   -6.35e+05   -4.62e+05
Grundskyldspromillen                                               5.925e+04   5065.408     11.697      0.000    4.93e+04    6.92e+04
Liggetid                                                           -413.0952    106.236     -3.888      0.000    -621.315    -204.876
m2                                                                   2.2e+04   1384.665     15.888      0.000    1.93e+04    2.47e+04
Grund                                                                21.1733      2.277      9.298      0.000      16.710      25.636
Ejerudgift                                                          829.2358     88.098      9.413      0.000     656.566    1001.906
Rooms                                                              -1.74e+04   3.15e+04     -0.552      0.581   -7.92e+04    4.44e+04
Build                                                               458.8275    449.601      1.021      0.307    -422.375    1340.030
Toilets                                                             6.06e+05   7.19e+04      8.430      0.000    4.65e+05    7.47e+05
Elevator                                                           1.049e+05   1.28e+05      0.817      0.414   -1.47e+05    3.57e+05
Altan                                                             -1.859e+05   8.59e+04     -2.165      0.030   -3.54e+05   -1.76e+04
Terasse                                                            6.993e+05   2.69e+05      2.597      0.009    1.72e+05    1.23e+06
antal anmeldelser                                                   693.4608     33.812     20.509      0.000     627.191     759.731
antal sigtelser                                                   -1987.5744    100.564    -19.764      0.000   -2184.675   -1790.473
Familiernes gennemsnitlige indkomst indkomst                          4.2553      0.819      5.195      0.000       2.650       5.861
m2 parker, sportsanlæg og rekreative omr pr indbygger             -1.238e+04   1000.857    -12.373      0.000   -1.43e+04   -1.04e+04
indbyggertal                                                         -3.7606      0.953     -3.948      0.000      -5.628      -1.894
a